# Overview
Online retailers face significant financial challenges due to high return rates, especially in industries like clothing, where return rates can be as high as 50%. While free or low-cost return shipping has become an expectation among customers, it places a substantial burden on businesses. To mitigate this, companies can leverage data-driven strategies to predict and reduce return rates.

Your task is to analyze the provided datasets and develop a predictive model that determines whether an ordered item will be returned. You will also extract actionable insights into the key factors influencing product returns.

# Dataset Description
You will be provided with two datasets:

N2N Train Set: Contains historical order data, including product details, user demographics, and whether the product was returned.

N2N Test Set: A separate dataset for evaluation, where you will apply your model to predict return probabilities.

The datasets contain the following attributes:

```
order_item_id: Unique identifier for each order.
order_date: Date the order was placed.
delivery_date: Date the product was delivered.
item_id: Unique identifier for the product.
item_size: Size of the ordered product.
item_color: Color of the ordered product.
brand_id: Unique identifier for the product's brand.
item_price: Price of the product.
user_id: Unique identifier for the customer.
user_title: Customer's salutation or title.
user_dob: Customer's date of birth.
user_state: Customer's state of residence.
user_reg_date: Date of customer registration on the platform.
return: Binary indicator of return status (0 = not returned, 1 = returned).
```

These fields provide comprehensive details on customer orders, product characteristics, and user demographics, enabling the identification of patterns associated with product returns.

---

## 1. Exploratory Data Analysis (EDA)

- Identify trends and patterns in product returns.

- Examine relationships between product attributes, user demographics, and return behavior.

In [95]:
# Other imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# Machine Learning Models
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier

# Model Evaluation
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error, r2_score


In [96]:
train_dataset_path = os.path.join("Dataset", "Dataset", "N2NTRAINSET.csv")
test_dataset_path = os.path.join("Dataset", "Dataset", "N2NTESTSET.csv")

train_df = pd.read_csv(train_dataset_path)
test_df = pd.read_csv(test_dataset_path)

print(f"{len(train_df.columns)} Columns: {train_df.columns}")
train_df

14 Columns: Index(['order_item_id', 'order_date', 'delivery_date', 'item_id', 'item_size',
       'item_color', 'brand_id', 'item_price', 'user_id', 'user_title',
       'user_dob', 'user_state', 'user_reg_date', 'return'],
      dtype='object')


,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
0,1,1/7/2016,NaN,247.0,39,black,40.0,49.90,182.0,Mrs,17-04-1961,1002.0,17-02-2015,0.0
1,2,1/7/2016,3/7/2016,24.0,39,brown,17.0,59.90,387.0,Mrs,29-07-1958,1002.0,16-02-2016,0.0
2,3,1/7/2016,3/7/2016,905.0,l,grey,33.0,59.90,387.0,Mrs,29-07-1958,1002.0,16-02-2016,0.0
3,4,1/7/2016,18-07-2016,9.0,8+,blue,4.0,59.95,393.0,Mrs,23-12-1963,1008.0,17-02-2015,0.0
4,5,1/7/2016,3/7/2016,55.0,42,blue,1.0,44.90,393.0,Mrs,23-12-1963,1008.0,17-02-2015,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60681,60682,9/9/2016,11/9/2016,1932.0,39,black,24.0,89.90,48045.0,Mrs,21-11-1949,1007.0,17-02-2015,0.0
60682,60683,9/9/2016,12/9/2016,1412.0,37,blue,54.0,119.90,48081.0,Mrs,30-09-1964,1004.0,16-12-2015,0.0
60683,60684,9/9/2016,8/10/2016,1415.0,m,green,3.0,24.90,48081.0,Mrs,30-09-1964,1004.0,16-12-2015,0.0
60684,60685,9/9/2016,12/9/2016,1622.0,20,blue,34.0,59.90,48081.0,Mrs,30-09-1964,1004.0,16-12-2015,0.0


As we can see, we have 12 features in our dataset (order_item_id is just an index, and return is the output value). Let's see what kind of values we have in each column:

In [97]:
train_df.drop(columns=["order_item_id"], inplace=True)

y_train = train_df["return"]
X_train = train_df.drop(columns=["return"])

In [98]:
import re

def date_is_dd_mm(x):
    if type(x) is str and int(re.split(r"[\-/]", x)[0]) > 12:
        return True
    else:
        return False
    
def date_is_mm_dd(x):
    if type(x) is str and int(re.split(r"[\-/]", x)[1]) > 12:
        return True
    else:
        return False
        

# For all unique dates, keep only those where the first number > 12
# After scanning the data, there are 2 formats: xx-xx-xxxx and xx/xx/xxxx
# With "-" I know it's DD-MM-YYYY, with "/" I'm not sure so I'm checking with this
for col in ['order_date', 'delivery_date', 'user_dob', 'user_reg_date']:
    mm_dd_dates = list(filter(date_is_mm_dd, set(X_train['order_date'])))
    dd_mm_dates = list(filter(date_is_dd_mm, set(X_train['order_date'])))

    # This is the list of dates we are CERTAIN is in DD-MM or MM-DD format
    print(f"MM-DD Dates: {mm_dd_dates}")
    print(f"DD-MM Dates: {dd_mm_dates}")

MM-DD Dates: []
DD-MM Dates: ['26-07-2016', '26-06-2016', '25-06-2016', '31-07-2016', '23-06-2016', '18-07-2016', '24-06-2016', '27-08-2016', '29-06-2016', '28-06-2016', '27-07-2016', '20-07-2016', '23-08-2016', '22-06-2016', '22-07-2016', '24-07-2016', '25-08-2016', '30-06-2016', '28-08-2016', '30-07-2016', '30-08-2016', '15-07-2016', '29-08-2016', '16-08-2016', '15-08-2016', '28-07-2016', '21-07-2016', '22-08-2016', '20-08-2016', '16-07-2016', '26-08-2016', '14-07-2016', '18-08-2016', '31-08-2016', '23-07-2016', '17-07-2016', '13-08-2016', '21-08-2016', '14-08-2016', '19-08-2016', '13-07-2016', '17-08-2016', '27-06-2016', '19-07-2016', '25-07-2016', '24-08-2016', '29-07-2016']
MM-DD Dates: []
DD-MM Dates: ['26-07-2016', '26-06-2016', '25-06-2016', '31-07-2016', '23-06-2016', '18-07-2016', '24-06-2016', '27-08-2016', '29-06-2016', '28-06-2016', '27-07-2016', '20-07-2016', '23-08-2016', '22-06-2016', '22-07-2016', '24-07-2016', '25-08-2016', '30-06-2016', '28-08-2016', '30-07-2016', '3

In [99]:
X_train

,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date
0,1/7/2016,NaN,247.0,39,black,40.0,49.90,182.0,Mrs,17-04-1961,1002.0,17-02-2015
1,1/7/2016,3/7/2016,24.0,39,brown,17.0,59.90,387.0,Mrs,29-07-1958,1002.0,16-02-2016
2,1/7/2016,3/7/2016,905.0,l,grey,33.0,59.90,387.0,Mrs,29-07-1958,1002.0,16-02-2016
3,1/7/2016,18-07-2016,9.0,8+,blue,4.0,59.95,393.0,Mrs,23-12-1963,1008.0,17-02-2015
4,1/7/2016,3/7/2016,55.0,42,blue,1.0,44.90,393.0,Mrs,23-12-1963,1008.0,17-02-2015
...,...,...,...,...,...,...,...,...,...,...,...,...
60681,9/9/2016,11/9/2016,1932.0,39,black,24.0,89.90,48045.0,Mrs,21-11-1949,1007.0,17-02-2015
60682,9/9/2016,12/9/2016,1412.0,37,blue,54.0,119.90,48081.0,Mrs,30-09-1964,1004.0,16-12-2015
60683,9/9/2016,8/10/2016,1415.0,m,green,3.0,24.90,48081.0,Mrs,30-09-1964,1004.0,16-12-2015
60684,9/9/2016,12/9/2016,1622.0,20,blue,34.0,59.90,48081.0,Mrs,30-09-1964,1004.0,16-12-2015


After checking all the date columns, we know for sure the xx-xx-xxxx dates are in the dd-mm-yyyy format. However, it seems the xx/xx/xxxx dates are ambiguous as they could be either day then month, or month then day. For now I will go with the assumption that the forward slash dates are mm/dd/yyyy, for a couple of reasons:

1. N2N is based in the US where we primarily use the mm/dd/yyyy format
2. The fact that there are two different separators could imply that they are different format

In [100]:
# Drop NaN values
X_train = X_train.dropna()

set(map(type, X_train['delivery_date']))

{str}

In [101]:
X_train.head()

,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date
1,1/7/2016,3/7/2016,24.0,39,brown,17.0,59.90,387.0,Mrs,29-07-1958,1002.0,16-02-2016
2,1/7/2016,3/7/2016,905.0,l,grey,33.0,59.90,387.0,Mrs,29-07-1958,1002.0,16-02-2016
3,1/7/2016,18-07-2016,9.0,8+,blue,4.0,59.95,393.0,Mrs,23-12-1963,1008.0,17-02-2015
4,1/7/2016,3/7/2016,55.0,42,blue,1.0,44.90,393.0,Mrs,23-12-1963,1008.0,17-02-2015
5,1/7/2016,18-07-2016,214.0,xxl,ash,49.0,44.90,393.0,Mrs,23-12-1963,1008.0,17-02-2015


In [102]:
# Convert string (mm/dd/yyyy or dd-mm-yyyy) to datetime
def convert_to_datetime(date):
    if "-" in date:
        return pd.to_datetime(date, format="%d-%m-%Y")
    else:
        return pd.to_datetime(date, format="%m/%d/%Y")
    
# Apply the function to the date columns
X_train["delivery_date"] = X_train["delivery_date"].apply(convert_to_datetime)
X_train["order_date"] = X_train["order_date"].apply(convert_to_datetime)
X_train["user_dob"] = X_train["user_dob"].apply(convert_to_datetime)
X_train["user_reg_date"] = X_train["user_reg_date"].apply(convert_to_datetime)

C:\Users\abhin\AppData\Local\Temp\ipykernel_32708\3679235857.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["delivery_date"] = X_train["delivery_date"].apply(convert_to_datetime)
C:\Users\abhin\AppData\Local\Temp\ipykernel_32708\3679235857.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["order_date"] = X_train["order_date"].apply(convert_to_datetime)
C:\Users\abhin\AppData\Local\Temp\ipykernel_32708\3679235857.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [109]:
print(set(map(type, X_train['order_date'])))
print(set(map(type, X_train['delivery_date'])))
print(set(map(type, X_train['user_dob'])))
print(set(map(type, X_train['user_reg_date'])))

# order_date, delivery_date, user_dob, user_reg_date are now datetime objects
X_train.info()

{<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
{<class 'pandas._libs.tslibs.timestamps.Timestamp'>}
<class 'pandas.core.frame.DataFrame'>
Index: 50061 entries, 1 to 60685
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_date     50061 non-null  datetime64[ns]
 1   delivery_date  50061 non-null  datetime64[ns]
 2   item_id        50061 non-null  float64       
 3   item_size      50061 non-null  object        
 4   item_color     50061 non-null  object        
 5   brand_id       50061 non-null  float64       
 6   item_price     50061 non-null  float64       
 7   user_id        50061 non-null  float64       
 8   user_title     50061 non-null  object        
 9   user_dob       50061 non-null  datetime64[ns]
 10  user_state     50061 non-null  float64       
 11  user_reg_date  5